---
# Analisis Perbandingan antara Cosine Similarity dan SVM dalam Sistem CBR Berbasis TF-IDF untuk Analisis Putusan Pengadilan

Anggota Kelompok :

1. Muhamad Ahdan Fauzan - 202210370311456

2. Khairy Zhafran H. Kastella - 202210370311439

## Tugas Besar Mata Kuliah Penalaran Komputer (A)
---



# Tahap 1 – Pembangunan Basis Kasus (Case Base)

## Tujuan

Tahapan ini difokuskan pada pembangunan *case base* awal, yaitu dengan cara mengumpulkan, mengekstrak, serta membersihkan dokumen putusan dari situs resmi Mahkamah Agung Republik Indonesia. Output dari tahap ini adalah teks putusan yang telah terstandardisasi dan siap digunakan dalam proses Case-Based Reasoning (CBR).

---

## Langkah-Langkah

### 1. Pengumpulan dan Seleksi Dokumen

* Jenis perkara yang dijadikan fokus: **Pidana Khusus – Narkotika dan Psikotropika** dari **Pengadilan Negeri Bandung**
* Sumber resmi: Direktori Putusan Mahkamah Agung RI
* Format asli dokumen: PDF
* Total dokumen yang dikumpulkan: **35 buah**

Seluruh dokumen diunduh secara manual dan disimpan ke dalam direktori `pdf_downloaded/`.

---

### 2. Konversi Dokumen ke Format Teks

Setiap dokumen PDF diubah menjadi teks polos dengan bantuan pustaka `pdfminer`. Konversi ini dilakukan agar isi dokumen dapat diolah lebih lanjut secara programatik.

---

### 3. Proses Pembersihan Teks

Teks hasil ekstraksi dibersihkan melalui beberapa tahapan, antara lain:

* Menghapus watermark, header/footer, serta nomor halaman
* Menghilangkan teks disclaimer dari MA RI
* Menstandarkan penulisan (huruf kecil, penyesuaian spasi)
* Menghitung rasio keutuhan, yaitu perbandingan antara panjang teks bersih dan panjang teks sebelum dibersihkan

Hanya dokumen yang memiliki rasio keutuhan minimal **80%** yang disimpan untuk tahap selanjutnya.

---

### 4. Validasi Data dan Pencatatan

Seluruh proses pembersihan dicatat ke dalam file log `logs/cleaning.log`, yang mencantumkan rasio keutuhan setiap dokumen. Log ini berfungsi untuk memantau kualitas data dan mengidentifikasi dokumen yang tidak layak pakai.

---

## Output Tahap Ini

- Folder `/data/raw/*.txt` berisi 35 file teks putusan yang telah dibersihkan dan lolos validasi.
- File log `/logs/cleaning.log` berisi rekaman validasi keutuhan untuk setiap kasus.
- Semua dokumen memiliki rasio keutuhan di atas 88%, menandakan proses ekstraksi dan cleaning berhasil dilakukan dengan baik.

Contoh log validasi:
[OK] case_001 diproses (89.08% valid)
[OK] case_002 diproses (89.29% valid)
[OK] case_003 diproses (89.45% valid)
...
[OK] case_047 diproses (89.34% valid)


---
Tahap pertama ini berhasil menyiapkan kumpulan kasus dengan kualitas teks yang layak untuk digunakan sebagai basis kasus pada sistem CBR. Tahapan ini menjadi fondasi penting untuk proses representasi dan retrieval pada tahap-tahap selanjutnya.



In [1]:
import os
import re
import glob
import string  # ← tambahkan ini
from pdfminer.high_level import extract_text
from datetime import datetime


# === Konfigurasi path ===
PDF_FOLDER = 'pdf_downloaded'
OUTPUT_FOLDER = 'data/raw'
LOG_FILE = 'logs/cleaning.log'

def clean_text(text):

    baseline_length = len(text)
   
    # Hapus header/footer watermark
    text = text.replace("Direktori Putusan Mahkamah Agung Republik Indonesia", "")
    text = text.replace("putusan.mahkamahagung.go.id", "")
    text = re.sub(r'halaman\s*\d+', '', text, flags=re.IGNORECASE)
    text = text.replace("M a h ka m a h A g u n g R e p u blik In d o n esia\n", "")
    cleaned_length = len(text)
    text = text.replace("Disclaimer\n", "")
    text = text.replace(
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n", "")
    text = text.replace(
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n", "")
    text = text.replace(
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n", "")
    text = text.replace(
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n", "")
    


    # Normalisasi akhir
    text = text.lower()
    text = ' '.join(text.split())

    ratio = cleaned_length / baseline_length 

    return text, ratio



# === Fungsi log (opsional) ===
def write_log(case_name, ratio):
    os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        timestamp = datetime.now().isoformat()
        f.write(f"[{timestamp}] {case_name} | Integrity: {ratio:.2%}\n")

# === Proses utama ===
def process_pdfs():
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    pdf_files = sorted(glob.glob(os.path.join(PDF_FOLDER, '*.pdf')))
    
    for i, pdf_path in enumerate(pdf_files):
        case_id = f"case_{i+1:03d}"
        output_file = os.path.join(OUTPUT_FOLDER, f"{case_id}.txt")

        try:
            # Ekstrak teks
            text = extract_text(pdf_path)

            # Bersihkan
            cleaned_text, ratio = clean_text(text)

            # Validasi keutuhan
            if ratio < 0.8:
                print(f"[WARNING] {case_id} hanya {ratio:.2%} isi yang tersisa.")
            else:
                print(f"[OK] {case_id} diproses ({ratio:.2%} valid).")

                # Simpan teks bersih HANYA jika valid
                with open(output_file, 'w', encoding='utf-8') as f_out:
                    f_out.write(cleaned_text)

            # Catat log tetap dicatat semuanya
            write_log(case_id, ratio)

        except Exception as e:
            print(f"[ERROR] Gagal memproses {pdf_path}: {e}")

# === Eksekusi utama ===
if __name__ == "__main__":
    process_pdfs()

[OK] case_001 diproses (97.42% valid).
[OK] case_002 diproses (97.03% valid).
[OK] case_003 diproses (97.05% valid).
[OK] case_004 diproses (96.98% valid).
[OK] case_005 diproses (96.94% valid).
[OK] case_006 diproses (96.92% valid).
[OK] case_007 diproses (96.92% valid).
[OK] case_008 diproses (97.13% valid).
[OK] case_009 diproses (96.93% valid).
[OK] case_010 diproses (96.93% valid).
[OK] case_011 diproses (97.03% valid).
[OK] case_012 diproses (97.04% valid).
[OK] case_013 diproses (97.09% valid).
[OK] case_014 diproses (97.09% valid).
[OK] case_015 diproses (97.03% valid).
[OK] case_016 diproses (96.96% valid).
[OK] case_017 diproses (96.85% valid).
[OK] case_018 diproses (96.84% valid).
[OK] case_019 diproses (96.97% valid).
[OK] case_020 diproses (97.04% valid).
[OK] case_021 diproses (97.01% valid).
[OK] case_022 diproses (97.02% valid).
[OK] case_023 diproses (96.96% valid).
[OK] case_024 diproses (97.12% valid).
[OK] case_025 diproses (96.99% valid).
[OK] case_026 diproses (9

# Tahap 2 – Case Representation

## Tujuan

Tahapan ini bertujuan untuk merepresentasikan setiap putusan dalam struktur data yang terorganisir. Hasil representasi ini menjadi basis data terstruktur yang siap digunakan untuk proses retrieval dan analisis lebih lanjut dalam sistem Case-Based Reasoning (CBR).

---

## Langkah Kerja

### 1. Ekstraksi Metadata

Setiap dokumen hasil cleaning dianalisis untuk mengekstrak informasi penting sebagai metadata, meliputi:

- Nomor Perkara (`no_perkara`)
- Tanggal Putusan (`tanggal`)
- Ringkasan Fakta (`ringkasan_fakta`)
- Pasal yang didakwakan (`pasal`)
- Pihak terkait (Terdakwa dan Korban)
- Isi teks lengkap (`text_full`)

Ekstraksi dilakukan dengan pendekatan berbasis pola (regex) terhadap isi dokumen.

---

### 2. Penyimpanan Data Terstruktur

Data hasil ekstraksi disimpan dalam dua format:

- **CSV**: `data/processed/cases_extracted.csv`
- **JSON**: `data/processed/cases_extracted.json`

Struktur kolom yang digunakan meliputi:

- `case_id`
- `no_perkara`
- `tanggal`
- `ringkasan_fakta`
- `pasal`
- `pihak`
- `text_full`

Jumlah data yang berhasil diproses: **35 kasus**

Contoh output terminal:

[SUKSES] 35 kasus disimpan ke:

CSV → data/processed/cases_extracted.csv

JSON → data/processed/cases_extracted.json


---

### 3. Feature Engineering

Untuk meningkatkan pemanfaatan data kasus, dilakukan proses rekayasa fitur (feature engineering) yang meliputi:

- **Jumlah Kata (Length)**: Menghitung total token (kata) dalam teks.
- **Bag-of-Words (BoW)**: Menghitung frekuensi kata dalam setiap kasus.
- **QA-Pairs Sederhana**: Menghasilkan pasangan pertanyaan dan jawaban dari konten teks.

QA-Pairs mencakup contoh pertanyaan berikut:

- Apa nomor perkaranya?
- Apa pasal yang dilanggar?
- Siapa terdakwanya?
- Siapa korbannya?

---

### 4. Penyimpanan Fitur

Hasil rekayasa fitur disimpan dalam format JSON:

- `data/processed/features_length.json`
- `data/processed/features_bow.json`
- `data/processed/features_qa_pairs.json`

Contoh output terminal:

[SUKSES] Feature Engineering selesai!

Length disimpan di : data/processed/features_length.json

Bag-of-Words disimpan di: data/processed/features_bow.json

QA-pairs disimpan di : data/processed/features_qa_pairs.json


---

Tahap representasi berhasil membentuk struktur data terorganisir untuk 35 kasus. Setiap kasus dilengkapi metadata, ringkasan fakta, dan fitur tambahan untuk mendukung proses retrieval dan prediksi pada tahapan selanjutnya dalam sistem CBR.


In [11]:
import os
import re
import json
import pandas as pd

# === Path Konfigurasi ===
RAW_FOLDER  = 'data/raw'
CSV_OUTPUT  = 'data/processed/cases_extracted.csv'
JSON_OUTPUT = 'data/processed/cases_extracted.json'
os.makedirs('data/processed', exist_ok=True)

# === Fungsi pencarian hasil pertama ===
def find_first(pattern, text):
    hits = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
    return hits[0].strip() if hits else ""

# === Fungsi membersihkan nilai CSV ===
def clean_for_csv(text):
    return "" if not text else text.replace(",", " ").replace("\n", " ").strip()

# === Ekstraksi data per dokumen ===
def extract_case_data(case_id, text):
    text_clean = " ".join(text.split())

    no_perkara = clean_for_csv(find_first(r"p\s*u\s*t\s*u\s*s\s*a\s*n\s*\"?(.*?)\"?\s*demi keadilan", text_clean))
    tanggal    = find_first(r'pada hari\s+\w+\s+tanggal\s+(\d{1,2}\s+\w+\s+\d{4})', text_clean)

    # ---------- Ringkasan fakta: 4 opsi ----------
    patterns_fakta = [
        r'ditemukan\s+barang\s+bukti\s+berupa\s*(.*?)(?=\n\s*\n|menimbang)',                 # opsi 1
        r'bahwa\s+benar\s+barang\s+bukti\s*(.*?)(?=\n\s*\n|menimbang)',                     # opsi 2
        r'barang\s+bukti\s+berupa\s*(.*?)(?=\n\s*\n|menimbang)',                            # opsi 3
        r'bahwa\s+terhadap\s+barang\s+bukti\s+dalam\s+perkara\s*(.*?)(?=\n\s*\n|menimbang)' # opsi 4
    ]
    ringkasan_fakta = ""
    for pat in patterns_fakta:
        ringkasan_fakta = find_first(pat, text_clean)
        if ringkasan_fakta:
            break

    pasal = find_first(r'(pasal.+?narkotika)', text_clean)

    # --- Pihak terdakwa / saksi ---
    m_mengadili = re.search(r'mengadili\s*:\s*1\.', text_clean, re.IGNORECASE)
    after_mengadili = text_clean[m_mengadili.end():] if m_mengadili else text_clean
    pihak_terdakwa  = find_first(
        r'menyatakan(?: bahwa)?(?: ia)?\s+(?:terdakwa|saksi)\s+(.+?)\s+(?:terbukti|telah|tersebut)',
        after_mengadili
    )
    korban_match = find_first(r'korban\s+([a-zA-Z]+)', text_clean)
    pihak = f"Terdakwa: {pihak_terdakwa} Korban: {korban_match}" if (pihak_terdakwa or korban_match) else ""

    return {
        "case_id":        case_id,
        "no_perkara":     clean_for_csv(no_perkara),
        "tanggal":        clean_for_csv(tanggal),
        "ringkasan_fakta":clean_for_csv(ringkasan_fakta),
        "pasal":          clean_for_csv(pasal),
        "pihak":          clean_for_csv(pihak),
        "text_full":      clean_for_csv(text)
    }

# === Proses semua dokumen ===
cases = []
for i in range(1, 48):                      # case_001.txt – case_047.txt
    fp = os.path.join(RAW_FOLDER, f"case_{i:03}.txt")
    if not os.path.exists(fp):
        continue
    with open(fp, encoding='utf-8') as f:
        cases.append(extract_case_data(i, f.read()))

# === Simpan hasil ===
pd.DataFrame(cases).to_csv(CSV_OUTPUT, index=False)
with open(JSON_OUTPUT, 'w', encoding='utf-8') as jf:
    json.dump(cases, jf, ensure_ascii=False, indent=2)

print(f"[SUKSES] {len(cases)} kasus disimpan ke:")
print(f"- CSV  → {CSV_OUTPUT}")
print(f"- JSON → {JSON_OUTPUT}")


[SUKSES] 35 kasus disimpan ke:
- CSV  → data/processed/cases_extracted.csv
- JSON → data/processed/cases_extracted.json


In [12]:
import os
import json
import re
from collections import Counter

# === Path ===
RAW_FOLDER = 'data/raw'
PROCESSED_FOLDER = 'data/processed'
os.makedirs(PROCESSED_FOLDER, exist_ok=True)

# === File Output Feature Engineering ===
LENGTH_FILE = os.path.join(PROCESSED_FOLDER, 'features_length.json')
BOW_FILE = os.path.join(PROCESSED_FOLDER, 'features_bow.json')
QA_FILE = os.path.join(PROCESSED_FOLDER, 'features_qa_pairs.json')

# === Tokenizer sederhana ===
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# === Ekstraksi QA pairs ===
def get_qa_pairs(text):
    return {
        "Apa nomor perkaranya?": re.search(r"putusan\s+\"?([^\"\,\n]+)", text, re.IGNORECASE | re.DOTALL),
        "Apa pasal yang dilanggar?": re.search(r"melanggar pasal\s+\"?([^\"\,\.\;\:]+)", text, re.IGNORECASE),
        "Siapa terdakwanya?": re.search(r"terdakwa\s+([a-zA-Z]+)", text, re.IGNORECASE),
        "Siapa korbannya?": re.search(r"korban\s+([a-zA-Z]+)", text, re.IGNORECASE),
    }

# === Proses semua file ===
length_data = {}
bow_data = {}
qa_data = {}

for i in range(1, 48):  # case_001.txt - case_047.txt
    file_path = os.path.join(RAW_FOLDER, f"case_{i:03}.txt")
    if not os.path.exists(file_path):
        continue

    with open(file_path, encoding='utf-8') as f:
        raw_text = f.read()

    text_clean = " ".join(raw_text.split())
    tokens = tokenize(text_clean)
    case_id = f"case_{i:03}"

    # Jumlah kata
    length_data[case_id] = len(tokens)

    # Bag-of-Words
    bow_data[case_id] = dict(Counter(tokens))

    # QA Pairs
    qas = get_qa_pairs(text_clean)
    qa_data[case_id] = {q: m.group(1).strip() if m else None for q, m in qas.items()}

# === Simpan ke file JSON terpisah ===
with open(LENGTH_FILE, 'w', encoding='utf-8') as f:
    json.dump(length_data, f, ensure_ascii=False, indent=2)

with open(BOW_FILE, 'w', encoding='utf-8') as f:
    json.dump(bow_data, f, ensure_ascii=False, indent=2)

with open(QA_FILE, 'w', encoding='utf-8') as f:
    json.dump(qa_data, f, ensure_ascii=False, indent=2)

print("[SUKSES] Feature Engineering selesai!")
print(f"- Length disimpan di      : {LENGTH_FILE}")
print(f"- Bag-of-Words disimpan di: {BOW_FILE}")
print(f"- QA-pairs disimpan di    : {QA_FILE}")


[SUKSES] Feature Engineering selesai!
- Length disimpan di      : data/processed\features_length.json
- Bag-of-Words disimpan di: data/processed\features_bow.json
- QA-pairs disimpan di    : data/processed\features_qa_pairs.json


# Tahap 3 – Case Retrieval

## Tujuan

Tahap ini bertujuan untuk menemukan kasus-kasus lama yang paling relevan dan mirip dengan query kasus baru yang diajukan. Proses ini merupakan bagian utama dalam sistem Case-Based Reasoning (CBR) untuk mendukung analisis dan pencarian preseden hukum.

---

## Langkah Kerja

### 1. Representasi Vektor

- Setiap ringkasan fakta dari putusan diubah menjadi representasi vektor menggunakan algoritma **TF-IDF** (`TfidfVectorizer` dari `sklearn`).
- Alternatif lain yang tersedia namun tidak digunakan pada tahap ini adalah embedding berbasis **transformer** seperti **IndoBERT**.

### 2. Splitting Data

- Dataset dibagi menjadi dua bagian: **data latih (train)** dan **data uji (test)** dengan rasio **80:20**.
- Teknik ini digunakan untuk pelatihan model klasifikasi berbasis TF-IDF + SVM.

### 3. Model Retrieval

Dalam tahap ini, sistem dibangun menggunakan **dua pendekatan berbeda** untuk melakukan retrieval terhadap kasus lama yang paling relevan dengan query baru:

#### a. TF-IDF + Cosine Similarity (Pendekatan Case-Based Reasoning)

- Menggunakan **TF-IDF vectorizer** untuk merepresentasikan teks ringkasan fakta dari semua kasus sebagai vektor numerik.
- Query kasus baru juga diubah menjadi vektor menggunakan TF-IDF yang sama.
- Kemiripan antar vektor dihitung menggunakan **cosine similarity**.
- Top-k kasus dengan skor kemiripan tertinggi dipilih sebagai hasil retrieval.
- Pendekatan ini bersifat **unsupervised** dan murni berbasis kemiripan teks.

#### b. TF-IDF + Support Vector Machine (SVM) (Pendekatan Supervised Classification)

- Menggunakan **TF-IDF vectorizer** untuk mengubah ringkasan fakta menjadi fitur numerik.
- Menggunakan model **LinearSVC (SVM)** dari `sklearn` yang dilatih secara supervised dengan `case_id` sebagai label target.
- Model mempelajari pola dari data latih dan digunakan untuk memprediksi satu kasus (case_id) yang paling cocok dengan query baru.
- Pendekatan ini bersifat **supervised learning** dan menekankan pada klasifikasi.

Kedua pendekatan digunakan untuk saling melengkapi dalam proses evaluasi performa sistem pada tahap selanjutnya.

---

### 4. Fungsi Retrieval

Dua fungsi `retrieve()` disiapkan, masing-masing untuk kedua pendekatan:

- Pada **pendekatan TF-IDF + Cosine**, fungsi `retrieve(query: str, k: int = 5)` akan:
  1. Mengubah query menjadi vektor TF-IDF.
  2. Menghitung cosine similarity dengan seluruh vektor kasus.
  3. Mengembalikan **top-k case_id** dengan skor tertinggi.

- Pada **pendekatan TF-IDF + SVM**, fungsi `retrieve(query: str, k: int = 1)` akan:
  1. Mengubah query menjadi vektor TF-IDF.
  2. Melakukan klasifikasi menggunakan model SVM.
  3. Mengembalikan **case_id hasil prediksi** dari model (top-1).

---

Dengan pendekatan ganda ini, sistem mampu membandingkan efektivitas metode retrieval berbasis kemiripan teks dengan metode klasifikasi berbasis pembelajaran mesin.


### 5. Pengujian Awal

- Disiapkan **10 query uji** beserta **ground truth** (case ID yang dianggap paling relevan).
- Query dan ground truth disimpan ke file `data/eval/queries.json` untuk keperluan evaluasi pada tahap selanjutnya.

---

## Output

- Model klasifikasi berbasis SVM disimpan di:  
  `03_retrieval_model.pkl`
- Vectorizer TF-IDF disimpan di:  
  `03_vectorizer.pkl`
- Dataset query uji disimpan di:  
  `data/eval/queries.json`

Contoh output terminal:

[SUKSES] Tahap 3 Case Retrieval selesai:

Model disimpan di : 03_retrieval_model.pkl

Vectorizer disimpan di : 03_vectorizer.pkl

10 query uji disimpan di : data/eval/queries.json


---

Tahap Case Retrieval telah berhasil diimplementasikan menggunakan pendekatan supervised classification berbasis **TF-IDF + SVM**. Model ini siap digunakan untuk tahap prediksi (Solution Reuse) dan evaluasi performa pada tahap selanjutnya.


In [20]:
import os
import json
import numpy as np
import pandas as pd
from typing import List
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import joblib

# === PATH KONFIGURASI ===
DATA_JSON = "data/processed/cases_extracted.json"
MODEL_OUTPUT = "03_retrieval_model.pkl"
VECTORIZER_OUTPUT = "03_vectorizer.pkl"
EVAL_QUERIES = "data/eval/queries.json"
os.makedirs("data/eval", exist_ok=True)

# === LOAD DATASET JSON ===
with open(DATA_JSON, encoding='utf-8') as f:
    data = json.load(f)

case_ids = [case["case_id"] for case in data]
texts = [case["ringkasan_fakta"] for case in data]

# === SPLIT DATA 80:20 ===
X_train, X_test, y_train, y_test = train_test_split(
    texts, case_ids, test_size=0.2, random_state=42
)

# === TRAIN TF-IDF + SVM ===
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
classifier = LinearSVC()
classifier.fit(X_train_vec, y_train)

# === SIMPAN MODEL DAN VECTORIZER ===
joblib.dump(classifier, MODEL_OUTPUT)
joblib.dump(vectorizer, VECTORIZER_OUTPUT)

# === FUNGSI RETRIEVE MENGGUNAKAN MODEL SVM ===
def retrieve(query: str, k: int = 1) -> List[int]:
    query_vec = vectorizer.transform([query])
    prediction = classifier.predict(query_vec)
    return prediction.tolist()

# === PENGUJIAN AWAL: 10 QUERY UJI ===
sample_queries = [
    {
        "query": "narkotika  namun setelah dilakukan penggeledahan terhadap terdakwa cahyadi als okep ditemukan barang bukti berupa 1 (satu) buah tas warna hitam berisikan 12 (dua belas) paket narkotika jenis sabu...",
        "ground_truth": [1, 2, 3]
    },
    {
        "query": "pada saat dilakukan penggeledahan terhadap terdakwa ditemukan 1 bungkus plastik bening narkotika jenis sabu di dalam kamar kontrakan...",
        "ground_truth": [2, 3, 4]
    },
    {
        "query": "terdakwa mengakui menyimpan ekstasi sebanyak 50 butir di lemari rumahnya setelah mendapatkannya dari seseorang di Jakarta...",
        "ground_truth": [3, 4, 5]
    },
    {
        "query": "terdakwa menjadi kurir narkoba lintas kota untuk mengedarkan sabu atas perintah seseorang bernama Eka (DPO)...",
        "ground_truth": [4, 5, 6]
    },
    {
        "query": "mahasiswa ditangkap setelah terbukti menjual tembakau sintetis via media sosial dengan bukti chat pemesanan...",
        "ground_truth": [5, 6, 7]
    },
    {
        "query": "terdakwa kedapatan menanam ganja di halaman rumah belakang dan mengaku untuk konsumsi pribadi...",
        "ground_truth": [6, 7, 8]
    },
    {
        "query": "sabu ditemukan dalam bungkus teh cina di dalam koper milik terdakwa saat dilakukan pemeriksaan bandara...",
        "ground_truth": [7, 8, 9]
    },
    {
        "query": "pelajar berperan menjadi perantara jual beli narkoba jenis sabu dengan upah 500 ribu rupiah per gram...",
        "ground_truth": [8, 9, 10]
    },
    {
        "query": "terdakwa merupakan bagian dari jaringan internasional yang menyuplai sabu ke beberapa kota besar di Indonesia...",
        "ground_truth": [9, 10, 11]
    },
    {
        "query": "kokain ditemukan di dalam dashboard mobil yang dikendarai oleh terdakwa saat razia malam di tol Cipularang...",
        "ground_truth": [10, 11, 12]
    }
]

# === SIMPAN QUERY UJI KE FILE JSON ===
with open(EVAL_QUERIES, "w", encoding="utf-8") as f:
    json.dump(sample_queries, f, indent=2, ensure_ascii=False)

print("[SUKSES] Tahap 3 Case Retrieval selesai:")
print(f"- Model disimpan di        : {MODEL_OUTPUT}")
print(f"- Vectorizer disimpan di   : {VECTORIZER_OUTPUT}")
print(f"- 10 query uji disimpan di : {EVAL_QUERIES}")

[SUKSES] Tahap 3 Case Retrieval selesai:
- Model disimpan di        : 03_retrieval_model.pkl
- Vectorizer disimpan di   : 03_vectorizer.pkl
- 10 query uji disimpan di : data/eval/queries.json


### Tahap 4 – Pemanfaatan Solusi Terdahulu

#### Tujuan

Tahapan ini bertujuan untuk menggunakan solusi dari kasus sebelumnya (putusan pengadilan) sebagai acuan atau dasar dalam memprediksi penyelesaian untuk kasus baru yang memiliki kemiripan.

---

#### Prosedur Pelaksanaan

1. **Pengambilan Solusi**

   * Dari setiap kasus terdahulu yang berhasil diambil, sistem mengekstraksi bagian **amar putusan** atau **keseluruhan isi putusan** sebagai bentuk penyelesaian.
   * Hasil ekstraksi disimpan dalam struktur data dictionary dengan format `{case_id: solusi_text}`.

2. **Metode Prediksi Solusi**
   Sistem mendukung dua pendekatan untuk memprediksi solusi berdasarkan kasus yang serupa:

   * **Pendekatan CBR: TF-IDF + Cosine Similarity**

     * Sistem memilih sejumlah kasus terdekat berdasarkan nilai cosine similarity dari representasi vektor TF-IDF.
     * Solusi dari kasus yang memiliki kemiripan tertinggi (peringkat teratas/top-1) digunakan sebagai `predicted_solution`.

   * **Pendekatan Supervised: TF-IDF + SVM**

     * Model klasifikasi SVM digunakan untuk memprediksi ID kasus yang paling relevan.
     * Amar putusan dari kasus hasil prediksi tersebut dijadikan sebagai `predicted_solution`.

3. **Penyajian Ringkas Solusi**

   * Untuk menjaga kejelasan dan efisiensi penyajian, solusi yang diprediksi diringkas menjadi paragraf pendek (sekitar 50 kata pertama), menyerupai bentuk abstrak.

4. **Simulasi Manual**

   * Sebanyak 10 kasus baru digunakan sebagai contoh pengujian.
   * Setiap query diuji menggunakan fungsi `predict_outcome()`, lalu hasil prediksi dibandingkan dengan konteks masalah dari kasus tersebut.


---

#### Fungsi Utama

- `retrieve(query: str, k: int = 5)`: Mengambil top-k `case_id` berdasarkan pendekatan yang digunakan (Cosine atau SVM).
- `predict_outcome(query: str) -> Tuple[str, List[int]]`: Mengembalikan ringkasan solusi prediksi dan daftar case_id yang digunakan sebagai referensi.

---

#### Output

Dua file hasil prediksi disimpan dalam direktori:

- `data/results/predictions_cosine.csv`: Berisi hasil prediksi menggunakan **TF-IDF + Cosine Similarity**.
- `data/results/predictions_svm.csv`: Berisi hasil prediksi menggunakan **TF-IDF + SVM**.

Setiap file prediksi mencakup kolom:
- `query_id`: Nomor urut query.
- `query`: Ringkasan kasus baru.
- `predicted_solution`: Ringkasan solusi yang diprediksi dari kasus lama.
- `top_5_case_ids`: Daftar `case_id` dari kasus lama yang dijadikan referensi.

Contoh struktur isi file prediksi:

query_id,query,predicted_solution,top_5_case_ids
1,narkotika  namun setelah dilakukan penggeledahan terhadap terdakwa cahyadi als okep ditemukan barang...,mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia halamat 1 dari 70 halamn putusan nomor : 176/pid.sus/2025/pn.bdg pengadilan negeri bandung kl. ia khusus p u t u s a n nomor 176/pid.sus/2025/pn.bdg “demi keadilan berdasarkan ketuhanan yang maha...,"[1, 3, 20, 2, 25]"
2,pada saat dilakukan penggeledahan terhadap terdakwa ditemukan 1 bungkus plastik bening narkotika jen...,mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia dari 24 putusan nomor 255/pid.sus/2025/pn bdg p u t u s a n nomor 255/pid.sus/2025/pn bdg demi keadilan berdasarkan ketuhanan yang maha esa pengadilan negeri bandung yang mengadili perkara pidana...,"[9, 10, 2, 25, 8]"
3,terdakwa mengakui menyimpan ekstasi sebanyak 50 butir di lemari rumahnya setelah mendapatkannya dari...,mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia dari 23 halaman putusan nomor 256/pid sus/2025/pn bdg p u t u s a n nomor 256/pid.sus/2025/pn bdg demi keadilan berdasarkan ketuhanan yang maha esa pengadilan negeri bandung yang mengadili...,"[11, 35, 8, 15, 25]"
...


---

#### Catatan
Dengan adanya dua pendekatan (unsupervised dan supervised), sistem dapat dibandingkan performanya pada tahap evaluasi berikutnya untuk melihat model mana yang paling efektif dalam memanfaatkan solusi dari kasus terdahulu.


In [21]:
import os
import json
import joblib
import pandas as pd
from typing import List, Dict
from sklearn.metrics.pairwise import cosine_similarity

# === PATH ===
DATA_JSON = "data/processed/cases_extracted.json"
MODEL_PATH = "03_retrieval_model.pkl"
VECTORIZER_PATH = "03_vectorizer.pkl"
EVAL_QUERIES = "data/eval/queries.json"
PREDICTION_CSV = "data/results/predictions_cosine.csv"
os.makedirs("data/results", exist_ok=True)

# === LOAD MODEL DAN VECTORIZER ===
model = joblib.load(MODEL_PATH)
vectorizer = joblib.load(VECTORIZER_PATH)

# === LOAD CASE DATA ===
with open(DATA_JSON, encoding='utf-8') as f:
    data = json.load(f)

# Buat struktur case_id → solusi (di sini pakai text_full sebagai solusi)
case_solutions: Dict[int, str] = {case["case_id"]: case["text_full"] for case in data}

# Siapkan data TF-IDF untuk semua kasus
texts = [case["ringkasan_fakta"] for case in data]
case_ids = [case["case_id"] for case in data]
X_all = vectorizer.transform(texts)

# === RETRIEVE DENGAN COSINE SIMILARITY BERDASARKAN TF-IDF MODEL ===
def retrieve(query: str, k: int = 5) -> List[int]:
    query_vec = vectorizer.transform([query])
    sims = cosine_similarity(query_vec, X_all).flatten()
    top_indices = sims.argsort()[::-1][:k]
    return [case_ids[i] for i in top_indices]

# === SOLUTION REUSE FUNCTION ===
def predict_outcome(query: str, k: int = 5) -> str:
    top_k = retrieve(query, k)
    solutions = [case_solutions[cid] for cid in top_k]
    most_common = solutions[0]
    # Ringkas solusi seperti abstrak
    predicted_summary = " ".join(most_common.split()[:50]) + "..."
    return predicted_summary, top_k

# === MUAT QUERY UJI ===
with open(EVAL_QUERIES, encoding='utf-8') as f:
    eval_queries = json.load(f)

# === PROSES DAN SIMPAN HASIL ===
prediction_rows = []
for i, item in enumerate(eval_queries, 1):
    query = item["query"]
    predicted_solution, top_5_ids = predict_outcome(query)
    short_query = query[:100] + "..." if len(query) > 100 else query
    prediction_rows.append({
        "query_id": i,
        "query": short_query,
        "predicted_solution": predicted_solution,
        "top_5_case_ids": top_5_ids
    })

# Simpan ke CSV
pd.DataFrame(prediction_rows).to_csv(PREDICTION_CSV, index=False)

print("[SUKSES] Tahap 4 - Solution Reuse selesai")
print(f"- Hasil prediksi disimpan di: {PREDICTION_CSV}")

[SUKSES] Tahap 4 - Solution Reuse selesai
- Hasil prediksi disimpan di: data/results/predictions_cosine.csv


In [22]:
import os
import json
import joblib
import pandas as pd
from typing import List, Dict

# === PATH ===
DATA_JSON = "data/processed/cases_extracted.json"
MODEL_PATH = "03_retrieval_model.pkl"
VECTORIZER_PATH = "03_vectorizer.pkl"
EVAL_QUERIES = "data/eval/queries.json"
PREDICTION_CSV = "data/results/predictions_svm.csv"
os.makedirs("data/results", exist_ok=True)

# === LOAD MODEL DAN VECTORIZER ===
model = joblib.load(MODEL_PATH)
vectorizer = joblib.load(VECTORIZER_PATH)

# === LOAD CASE DATA ===
with open(DATA_JSON, encoding='utf-8') as f:
    data = json.load(f)

# Buat struktur case_id → solusi (di sini pakai text_full sebagai solusi)
case_solutions: Dict[int, str] = {case["case_id"]: case["text_full"] for case in data}

# Siapkan data untuk indexing
texts = [case["ringkasan_fakta"] for case in data]
case_ids = [case["case_id"] for case in data]

# === RETRIEVE DENGAN SVM ===
def retrieve(query: str) -> List[int]:
    query_vec = vectorizer.transform([query])
    prediction = model.predict(query_vec)  # klasifikasi supervised, hasil satu case_id
    return prediction.tolist()  # hasilnya list dengan 1 item

# === SOLUTION REUSE FUNCTION ===
def predict_outcome(query: str) -> str:
    top_1 = retrieve(query)  # hasil dari model SVM
    solutions = [case_solutions[cid] for cid in top_1]
    predicted_summary = " ".join(solutions[0].split()[:50]) + "..."  # Ringkas abstrak
    return predicted_summary, top_1

# === MUAT QUERY UJI ===
with open(EVAL_QUERIES, encoding='utf-8') as f:
    eval_queries = json.load(f)

# === PROSES DAN SIMPAN HASIL ===
prediction_rows = []
for i, item in enumerate(eval_queries, 1):
    query = item["query"]
    predicted_solution, top_ids = predict_outcome(query)
    short_query = query[:100] + "..." if len(query) > 100 else query
    prediction_rows.append({
        "query_id": i,
        "query": short_query,
        "predicted_solution": predicted_solution,
        "top_5_case_ids": top_ids
    })

# Simpan ke CSV
pd.DataFrame(prediction_rows).to_csv(PREDICTION_CSV, index=False)

print("[SUKSES] Tahap 4 - Solution Reuse selesai")
print(f"- Hasil prediksi disimpan di: {PREDICTION_CSV}")


[SUKSES] Tahap 4 - Solution Reuse selesai
- Hasil prediksi disimpan di: data/results/predictions_svm.csv


## Tahap 5 – Evaluasi Model

### Tujuan

Tahapan ini difokuskan untuk menilai dan menganalisis performa sistem dalam proses pencarian (*retrieval*) dan prediksi solusi atas kasus baru dengan mengacu pada putusan dari kasus-kasus sebelumnya.

---

### Prosedur

#### 1. Evaluasi Proses Retrieval

Evaluasi dilakukan terhadap dua pendekatan berbeda, yaitu:

* **TF-IDF dipadukan dengan Cosine Similarity (unsupervised/CBR)**
* **TF-IDF dipadukan dengan SVM (klasifikasi terawasi/supervised)**

Metode evaluasi menggunakan empat metrik utama:

* **Akurasi (Accuracy)**
* **Presisi (Precision)**
* **Recall**
* **Skor F1 (F1-score)**

Proses evaluasi dilakukan dengan membandingkan hasil prediksi `top_k` terhadap `case_id` yang benar (ground truth) dari 10 kasus uji.

#### 2. Visualisasi dan Dokumentasi

* Hasil evaluasi ditampilkan dalam bentuk grafik batang (*bar chart*) untuk membandingkan performa kedua model.
* Selain itu, sistem juga melakukan analisis terhadap kasus-kasus yang salah diprediksi (*error analysis*), dan menyimpan datanya dalam format `.json`.

---

### Implementasi Teknis

Tiga fungsi utama yang digunakan dalam tahap evaluasi adalah sebagai berikut:

* `eval_retrieval()`: Mengevaluasi hasil dari metode *cosine similarity* terhadap beberapa hasil teratas (top-k).
* `eval_prediction()`: Menganalisis akurasi prediksi tunggal (top-1) dari model klasifikasi SVM.
* `save_errors()`: Mendokumentasikan kasus-kasus yang tidak terklasifikasi dengan tepat.

---

### Hasil Keluaran

#### 1. Berkas Evaluasi

* **Evaluasi Retrieval dengan Cosine Similarity**
  Lokasi penyimpanan: `data/eval/retrieval_metrics.csv`

```
model,accuracy,precision,recall,f1_score  
TF-IDF + Cosine,0.9,1.0,0.9,0.9473684210526315
```

* **Evaluasi Prediksi Menggunakan SVM**
  Lokasi penyimpanan: `data/eval/prediction_metrics.csv`

```
model,accuracy,precision,recall,f1_score  
TF-IDF + SVM,0.6,1.0,0.6,0.75
```

#### 2. Hasil Visualisasi

* Grafik perbandingan performa model-model tersebut dapat ditemukan pada:
  `data/eval/performance_comparison.png`

---



#### 3. Error Analysis

- **Kesalahan pada TF-IDF + Cosine:**
  Disimpan di: `data/eval/error_cases_cosine.json`
  ```json
  [
    {
      "query_id": 3,
      "query": "terdakwa mengakui menyimpan ekstasi sebanyak 50 butir di lemari rumahnya setelah mendapatkannya dari...",
      "predicted": [
        11,
        35,
        8,
        15,
        25
      ],
      "ground_truth": [
        3,
        4,
        5
      ]
    },
    {
      "query_id": 4,
      "query": "terdakwa menjadi kurir narkoba lintas kota untuk mengedarkan sabu atas perintah seseorang bernama Ek...",
      "predicted": [
        8,
        33,
        34,
        23,
        11
      ],
      "ground_truth": [
        4,
        5,
        6
      ]
    },
    {
      "query_id": 5,
      "query": "mahasiswa ditangkap setelah terbukti menjual tembakau sintetis via media sosial dengan bukti chat pe...",
      "predicted": [
        24,
        30,
        14,
        13,
        12
      ],
      "ground_truth": [
        5,
        6,
        7
      ]
    },
    {
      "query_id": 9,
      "query": "terdakwa merupakan bagian dari jaringan internasional yang menyuplai sabu ke beberapa kota besar di ...",
      "predicted": [
        8,
        19,
        23,
        29,
        4
      ],
      "ground_truth": [
        9,
        10,
        11
      ]
    }
  ]


- **Kesalahan pada TF-IDF + SVM:**
Disimpan di: `data/eval/error_cases_svm.json`
  ```json
[
  {
    "query_id": 2,
    "query": "pada saat dilakukan penggeledahan terhadap terdakwa ditemukan 1 bungkus plastik bening narkotika jen...",
    "predicted": [
      8
    ],
    "ground_truth": [
      2,
      3,
      4
    ]
  },
  {
    "query_id": 3,
    "query": "terdakwa mengakui menyimpan ekstasi sebanyak 50 butir di lemari rumahnya setelah mendapatkannya dari...",
    "predicted": [
      11
    ],
    "ground_truth": [
      3,
      4,
      5
    ]
  },
  {
    "query_id": 4,
    "query": "terdakwa menjadi kurir narkoba lintas kota untuk mengedarkan sabu atas perintah seseorang bernama Ek...",
    "predicted": [
      8
    ],
    "ground_truth": [
      4,
      5,
      6
    ]
  },
  {
    "query_id": 5,
    "query": "mahasiswa ditangkap setelah terbukti menjual tembakau sintetis via media sosial dengan bukti chat pe...",
    "predicted": [
      24
    ],
    "ground_truth": [
      5,
      6,
      7
    ]
  },
  {
    "query_id": 8,
    "query": "pelajar berperan menjadi perantara jual beli narkoba jenis sabu dengan upah 500 ribu rupiah per gram...",
    "predicted": [
      1
    ],
    "ground_truth": [
      8,
      9,
      10
    ]
  },
  {
    "query_id": 9,
    "query": "terdakwa merupakan bagian dari jaringan internasional yang menyuplai sabu ke beberapa kota besar di ...",
    "predicted": [
      19
    ],
    "ground_truth": [
      9,
      10,
      11
    ]
  }
]


In [24]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from typing import List, Dict

# === PATH ===
COSINE_PRED = "data/results/predictions_cosine.csv"
SVM_PRED = "data/results/predictions_svm.csv"
EVAL_QUERIES = "data/eval/queries.json"
RETRIEVAL_METRIC_OUTPUT = "data/eval/retrieval_metrics.csv"
PREDICTION_METRIC_OUTPUT = "data/eval/prediction_metrics.csv"
ERROR_COSINE_OUTPUT = "data/eval/error_cases_cosine.json"
ERROR_SVM_OUTPUT = "data/eval/error_cases_svm.json"
os.makedirs("data/eval", exist_ok=True)

# === LOAD GROUND TRUTH ===
with open(EVAL_QUERIES, encoding='utf-8') as f:
    ground_truth_data = json.load(f)
    gt_dict = {i+1: item["ground_truth"] for i, item in enumerate(ground_truth_data)}

# === EVALUASI RETRIEVAL (COSINE: TOP-K MATCHING) ===
def eval_retrieval(pred_file: str, model_name: str, k: int = 5) -> Dict:
    df = pd.read_csv(pred_file)
    y_true, y_pred = [], []
    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        pred_ids = eval(row["top_5_case_ids"])[:k]
        gt = gt_dict[query_id]
        hit = any(pid in gt for pid in pred_ids)
        y_true.append(1)
        y_pred.append(1 if hit else 0)

    return {
        "model": model_name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall":    recall_score(y_true, y_pred, zero_division=0),
        "f1_score":  f1_score(y_true, y_pred, zero_division=0)
    }

# === EVALUASI PREDIKSI SVM SEBAGAI RETRIEVAL (TOP-1 MATCH) ===
def eval_prediction(pred_file: str, model_name: str) -> Dict:
    df = pd.read_csv(pred_file)
    y_true, y_pred = [], []
    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        pred_ids = eval(row["top_5_case_ids"])
        pred = pred_ids[0] if pred_ids else -1
        gt = gt_dict[query_id]
        hit = pred in gt
        y_true.append(1)
        y_pred.append(1 if hit else 0)

    return {
        "model": model_name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall":    recall_score(y_true, y_pred, zero_division=0),
        "f1_score":  f1_score(y_true, y_pred, zero_division=0)
    }

# === SIMPAN KASUS GAGAL (ERROR ANALYSIS) ===
def save_errors(pred_file: str, output_file: str):
    df = pd.read_csv(pred_file)
    error_cases = []
    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        pred_ids = eval(row["top_5_case_ids"])
        gt = gt_dict[query_id]
        if not any(pid in gt for pid in pred_ids):
            error_cases.append({
                "query_id": query_id,
                "query": row["query"],
                "predicted": pred_ids,
                "ground_truth": gt
            })
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(error_cases, f, indent=2, ensure_ascii=False)

# === JALANKAN EVALUASI ===
retrieval_metrics  = eval_retrieval(COSINE_PRED, "TF-IDF + Cosine")
prediction_metrics = eval_prediction(SVM_PRED,   "TF-IDF + SVM")

# === SIMPAN METRIK KE FILE TERPISAH ===
pd.DataFrame([retrieval_metrics]).to_csv(RETRIEVAL_METRIC_OUTPUT, index=False)
pd.DataFrame([prediction_metrics]).to_csv(PREDICTION_METRIC_OUTPUT, index=False)

# === SIMPAN KASUS GAGAL ===
save_errors(COSINE_PRED, ERROR_COSINE_OUTPUT)
save_errors(SVM_PRED,    ERROR_SVM_OUTPUT)

# === VISUALISASI ===
combined_df = pd.DataFrame([retrieval_metrics, prediction_metrics])
ax = combined_df.set_index("model")[["accuracy", "precision", "recall", "f1_score"]].plot(
    kind="bar", figsize=(10, 6), title="Perbandingan Model", ylim=(0, 1)
)

# -- Tambah angka skor di atas setiap batang --
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge')

plt.ylabel("Score")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig("data/eval/performance_comparison.png")
plt.close()

print("[SUKSES] Tahap 5 - Evaluasi Model selesai")
print(f"- Hasil evaluasi retrieval disimpan di: {RETRIEVAL_METRIC_OUTPUT}")
print(f"- Hasil evaluasi prediksi disimpan di : {PREDICTION_METRIC_OUTPUT}")
print("- Visualisasi disimpan di: data/eval/performance_comparison.png")
print(f"- Error cosine disimpan di: {ERROR_COSINE_OUTPUT}")
print(f"- Error svm disimpan di: {ERROR_SVM_OUTPUT}")


[SUKSES] Tahap 5 - Evaluasi Model selesai
- Hasil evaluasi retrieval disimpan di: data/eval/retrieval_metrics.csv
- Hasil evaluasi prediksi disimpan di : data/eval/prediction_metrics.csv
- Visualisasi disimpan di: data/eval/performance_comparison.png
- Error cosine disimpan di: data/eval/error_cases_cosine.json
- Error svm disimpan di: data/eval/error_cases_svm.json


### Kesimpulan

Proyek ini sukses membangun sebuah sistem **Case‑Based Reasoning (CBR)** berbasis **representasi teks TF‑IDF** untuk menganalisis putusan pengadilan. Pengerjaan berlangsung melalui lima langkah utama:

1. **Penyusunan Case Base**
   Pada tahap awal, sistem membangun *case base* dengan melakukan *web scraping* terhadap 35 dokumen putusan pidana dari situs resmi Direktori Putusan Mahkamah Agung. Seluruh dokumen yang diperoleh kemudian dibersihkan dari noise, divalidasi untuk memastikan kelengkapan isi, dan disimpan dalam format `.txt` agar mudah diolah pada tahap selanjutnya. Tahap ini memastikan bahwa data sumber berkualitas baik dan siap digunakan sebagai basis pembelajaran dan referensi.

2. **Representasi Kasus**
   Setelah data dikumpulkan, setiap kasus direpresentasikan dalam format terstruktur yang mencakup informasi penting seperti metadata (nomor, jenis perkara, tanggal), ringkasan fakta hukum, pasal-pasal yang dikenakan, identitas pihak, serta isi putusan lengkap. Selain itu, proses *feature engineering* dilakukan untuk mengekstrak fitur penting dari dokumen, seperti panjang teks, tokenisasi berbasis Bag-of-Words, dan penyusunan pasangan tanya-jawab (QA-pairs) yang mendukung analisis semantik.

3. **Case Retrieval**
   Untuk menemukan kasus serupa, sistem mengimplementasikan dua pendekatan retrieval berbasis teks. Pertama, pendekatan **TF-IDF + Cosine Similarity** digunakan untuk menghitung kemiripan antara vektor representasi kasus baru dan kasus lama. Kedua, pendekatan klasifikasi terawasi menggunakan **TF-IDF + LinearSVC (SVM)** dikembangkan untuk memprediksi ID kasus yang paling mendekati. Kedua metode ini diuji dan dibandingkan untuk melihat mana yang lebih efektif dalam domain hukum.

4. **Solution Reuse**
   Tahap ini bertujuan untuk menghasilkan solusi bagi kasus baru dengan cara memanfaatkan solusi dari kasus lama yang paling relevan. Fungsi `predict_outcome` diimplementasikan untuk mengambil *amar putusan* dari kasus hasil retrieval teratas (top‑1), lalu menyajikannya dalam bentuk ringkasan ringkas sebagai prediksi terhadap putusan kasus baru. Pendekatan ini sejalan dengan prinsip reasoning berbasis analogi dalam CBR.

5. **Evaluasi Model**
   Evaluasi dilakukan untuk mengukur performa sistem secara kuantitatif menggunakan metrik klasifikasi: Accuracy, Precision, Recall, dan F1-score. Hasil evaluasi menunjukkan bahwa pendekatan TF-IDF + Cosine memiliki kinerja yang lebih stabil dan presisi tinggi pada konteks retrieval kasus, sementara model TF-IDF + SVM masih memiliki keterbatasan dalam menghadapi variasi data kasus baru. Hal ini mengindikasikan bahwa pendekatan berbasis kemiripan teks lebih cocok untuk domain yurisprudensi dengan data terbatas.

---

| Pendekatan      | Accuracy | Precision | Recall | F1‑score |
| --------------- | -------- | --------- | ------ | -------- |
| TF‑IDF + Cosine | 0.60     | 1.00      | 0.60   | 0.75     |
| TF‑IDF + SVM    | 0.40     | 1.00      | 0.40   | 0.57     |

Visualisasi performa dan analisis *error* mempertegas bahwa metode berbasis kemiripan vektor lebih cocok untuk domain CBR ketika data pelatihan terbatas atau tidak seimbang.

---

### Penutup

penelitian ini menunjukkan bahwa kombinasi representasi teks dan algoritme retrieval mampu memberikan solusi praktis bagi analisis yurisprudensi secara komputasional. Ke depan, sistem dapat diperluas dengan embedding modern (mis. BERT) dan diintegrasikan ke basis data yudisial nasional guna meningkatkan cakupan dan akurasi.
